# upload dataset and import pytorch 


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving new_data.csv.zip to new_data.csv.zip
User uploaded file "new_data.csv.zip" with length 117841345 bytes


In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch


tcmalloc: large alloc 1073750016 bytes == 0x584a2000 @  0x7f1e588322a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [0]:
import torch

In [0]:

! unzip new_data.csv.zip

Archive:  new_data.csv.zip
  inflating: new_data.csv            
   creating: __MACOSX/
  inflating: __MACOSX/._new_data.csv  


In [0]:
!ls


__MACOSX  new_data.csv	new_data.csv.zip  sample_data


In [0]:
import torch
import pandas as pd
df = pd.read_csv("new_data.csv")


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,31,40,43,44,45,48,49,61,68,69,70,75,77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:

df1 = df.loc[df['totals_transactionRevenue'] == 0]
dfx = df1.sample(20000)
df2 = df.loc[df['y'] > 0]
data = pd.concat([dfx,df2])

In [0]:
# len(df2)

18514

# add column: year month weekday

In [0]:
import pandas as pd
# data = pd.read_csv("new_data.csv")
df = data
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
# get new year feature
df['year'] = df['date'].dt.year
# get new month feature
df['month'] = df['date'].dt.month
# get new week feature 
df['week'] = df['date'].dt.week
# get new weekday feature 
df['weekday'] = df['date'].dt.weekday
# drop 'date'
del(df['date'])


In [0]:
# real.dt.second
realtime = pd.to_datetime(df['visitStartTime'], unit='s')
real = pd.to_datetime(realtime, format='%Y%m%d%H:%M:%s')
df['hour'] = real.dt.hour
df['minute'] = real.dt.minute
df['second'] = real.dt.second

In [0]:
import numpy as np
import pandas
# data = pandas.read_csv("new_data.csv")
data = df
data = data.replace(np.nan,0)
str_cols = ["channelGrouping", "device_browser", 
            "device_deviceCategory", "device_operatingSystem", 
            "geoNetwork_city", "geoNetwork_continent", 
            "geoNetwork_country", "geoNetwork_metro",
            "geoNetwork_networkDomain", "geoNetwork_region", 
            "geoNetwork_subContinent", "customDimensions_value"]
# str_cols = ["channelGrouping", 
#             "geoNetwork_city", "geoNetwork_continent", 
#             "geoNetwork_country", "geoNetwork_metro",
#             "geoNetwork_networkDomain", "geoNetwork_region", 
#             "geoNetwork_subContinent", "customDimensions_value"]
data[str_cols] = str(data[str_cols])
num_cols = ["year", "month","weekday","hour","minute","second","totals_hits", "totals_pageviews", "visitNumber", 'totals_bounces',  'totals_newVisits',
            'totals_transactions'] 
# num_cols = ["totals_hits", "totals_pageviews", "visitNumber", 'totals_bounces',  'totals_newVisits'] 


import torch.nn as nn
import torch.nn.functional
import torch.optim
import numpy.random
import math
import pandas
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder


# data["totals_transactionRevenue"] = data["totals_transactionRevenue"].apply(lambda x: math.log(1 + x))
# data["totals_transactionRevenue"] = data["y"]
    
label_encoders = {}
for str_col in str_cols:
    label_encoders[str_col] = LabelEncoder()
    data[str_col] = label_encoders[str_col].fit_transform(data[str_col].astype(str))

# training_data = df
msk = numpy.random.rand(len(data)) < 0.8
training_data = data[msk]
testing_data = data[~msk]
testing_id = testing_data['fullVisitorId']
target = training_data['totals_transactionRevenue']
training_data = training_data[str_cols + num_cols]   


#create the optimizer

import torch.utils.data as data_utils
training_data = training_data.values.astype('float')
training_data = torch.cuda.FloatTensor(training_data)
target = torch.cuda.FloatTensor(target.values)

loss_list = []


training_data = training_data.unsqueeze(0)
training_data = training_data.permute(1,0,2)
target = target.unsqueeze(0)
target = target.permute(1,0)
training_data = training_data.cuda()
target = target.cuda()
train = data_utils.TensorDataset(training_data, target)


In [0]:
!ls

adc.json  drive  __MACOSX  new_data.csv  sample_data


In [0]:
df['totals_transactions']

# CNN model and training process

In [0]:
import torch.nn as nn
import torch.nn.functional
import torch.optim
import numpy.random
import math
import pandas
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

import torch.utils.data as data_utils



class CNN_net(nn.Module):
    def __init__(self,str_cols, num_cols):
        
        super(CNN_net, self).__init__()
        
        self.fc1 = nn.Sequential(

            nn.Conv1d(1, 16, kernel_size=3),
            nn.BatchNorm1d(16),
            nn.MaxPool1d(2,2),
            nn.ReLU(),
            )
        self.fc2 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=3),
            nn.BatchNorm1d(32),
#             nn.MaxPool1d(2,2),
            nn.ReLU(),
            )
#         self.fc3 = nn.Dropout(0.5)
#         self.fc4 = nn.Linear((len(str_cols) + len(num_cols) - 4 ) * 32,1)
        self.fc4 = nn.Linear(288,1)
#         self.fc5 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        out = x.reshape(x.size(0), -1)
#         print(out.size())
#         out = self.fc3(out)
        f = self.fc4(out)
#         f = self.fc5(f)
        return f


net = CNN_net(str_cols, num_cols)
net = net.cuda()
print(net )

#zero the gradient buffers
net.zero_grad()
loss_list = []

epoch = 600

#create the optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=51e-5)
criterion = nn.MSELoss()

for i in range(epoch):
#     if i%100 == 0:
    print(i)
    torch.cuda.empty_cache()
    running_loss = 0
    train_loader = data_utils.DataLoader(train, batch_size=256, shuffle=True)
    for trainx,labelx in train_loader:
        #print(trainx.size())
        optimizer.zero_grad()
        loss = criterion(net(trainx), labelx)
        loss_list.append(loss)
        loss.backward()
        optimizer.step() 
        running_loss += float(loss.item())
        loss_list.append(running_loss)
    print(running_loss)


# label = testing_data['totals_transactionRevenue']
# label = torch.Tensor(label.values)
# testing_data = testing_data[str_cols + num_cols]
# testing_data = torch.Tensor(testing_data.values)
# output = net(testing_data)
# print(output)

CNN_net(
  (fc1): Sequential(
    (0): Conv1d(1, 16, kernel_size=(3,), stride=(1,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
  )
  (fc2): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (fc4): Linear(in_features=288, out_features=1, bias=True)
)
0
12712.657859802246
1
8450.831382751465
2
5157.634616851807
3
3919.531192779541
4
3340.123092651367
5
2908.3716678619385
6
2571.125816345215
7
2366.464701652527
8
2260.821078300476
9
2192.645215988159
10
2118.938940048218
11
2111.7259845733643
12
2117.964104652405
13
2065.397247314453
14
2055.5343685150146
15
2073.8960571289062
16
2029.6182279586792
17
2030.8668775558472
18
2013.7340078353882
19
2007.3999166488647
20
2020.4941129684448
21
2000.1461362838745
22
2003.63457

In [0]:
output

tensor([[-0.1241],
        [ 0.0549],
        [ 0.1801],
        ...,
        [16.7180],
        [17.2316],
        [16.9696]], device='cuda:0', grad_fn=<ThAddmmBackward>)

# initial loss

In [0]:
net.eval()
label = testing_data['totals_transactionRevenue']
label = torch.cuda.FloatTensor(label.values)
testing = testing_data[str_cols + num_cols]
testing = torch.cuda.FloatTensor(testing.values)


testing = testing.unsqueeze(0)
testing = testing.permute(1,0,2)
label = label.unsqueeze(0)
label = label.permute(1,0)
output = net(testing)
criterion = torch.nn.MSELoss()
loss = torch.sqrt(criterion(output, label))
loss

tensor(0.8212, device='cuda:0', grad_fn=<SqrtBackward>)

# calculate the final cumulative loss (loss of the the predition for user's whole spending)

In [0]:
import numpy as np
c = output.cpu().detach().numpy()
c = np.exp(c) -1
c1 = pandas.DataFrame(c)
c1.columns = ['pred']

cc = testing_id.to_frame().values
cc = pandas.DataFrame(cc)
cc.columns = ['id']

z = pandas.concat([c1,cc], axis=1, join='inner')
# x1 = z.groupby('id','pred').sum()
x1 = z.groupby(['id'])['pred'].agg('sum').reset_index()
x1['pred'] = x1['pred'] .apply(lambda x: math.log(1 + x))


c = label.cpu().detach().numpy()
c = np.exp(c) -1
c1 = pandas.DataFrame(c)
c1.columns = ['label']

cc = testing_id.to_frame().values
cc = pandas.DataFrame(cc)
cc.columns = ['id']

z = pandas.concat([c1,cc], axis=1, join='inner')

# x2 = z.groupby('id').agg(['sum'])
x2 = z.groupby(['id'])['label'].agg('sum').reset_index()
x2['label'] = x2['label'] .apply(lambda x: math.log(1 + x))
# x2['sum'] = s2['sum'].apply(lambda x: math.log(1 + x))
net.eval()
labels = x2['label']
labels = torch.cuda.FloatTensor(labels.values)
outs = x1['pred']
outs = torch.cuda.FloatTensor(outs.values)
criterion = torch.nn.MSELoss()
loss = torch.sqrt(criterion(outs, labels))



In [0]:
loss

tensor(0.8074, device='cuda:0')

#import model

In [0]:
import torch.nn as nn
import torch.nn.functional
import torch.optim
import numpy.random
import math
import pandas
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

import torch.utils.data as data_utils



class CNN_net(nn.Module):
    def __init__(self,str_cols, num_cols):
        
        super(CNN_net, self).__init__()
        
        self.fc1 = nn.Sequential(

            nn.Conv1d(1, 16, kernel_size=3),
            nn.BatchNorm1d(16),
            nn.MaxPool1d(2,2),
            nn.ReLU(),
            )
        self.fc2 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=3),
            nn.BatchNorm1d(32),
#             nn.MaxPool1d(2,2),
            nn.ReLU(),
            )
#         self.fc3 = nn.Dropout(0.5)
#         self.fc4 = nn.Linear((len(str_cols) + len(num_cols) - 4 ) * 32,1)
        self.fc4 = nn.Linear(288,1)
#         self.fc5 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        out = x.reshape(x.size(0), -1)
#         print(out.size())
#         out = self.fc3(out)
        f = self.fc4(out)
#         f = self.fc5(f)
        return f


# net = torch.load('./net')

In [0]:
import torch

In [0]:
net = torch.load('drive/data/net')

In [0]:

import torch
import pandas as pd
df = pd.read_csv("new_data.csv")
# data = pandas.read_csv("new_data.csv")
import pandas as pd
# data = pd.read_csv("new_data.csv")
s = df
df = s.sample(1000000)
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
# get new year feature
df['year'] = df['date'].dt.year
# get new month feature
df['month'] = df['date'].dt.month
# get new week feature 
df['week'] = df['date'].dt.week
# get new weekday feature 
df['weekday'] = df['date'].dt.weekday
# drop 'date'
del(df['date'])
# real.dt.second
realtime = pd.to_datetime(df['visitStartTime'], unit='s')
real = pd.to_datetime(realtime, format='%Y%m%d%H:%M:%s')
df['hour'] = real.dt.hour
df['minute'] = real.dt.minute
df['second'] = real.dt.second
import numpy as np
import pandas
data = df
data = data.replace(np.nan,0)
str_cols = ["channelGrouping", "device_browser", 
            "device_deviceCategory", "device_operatingSystem", 
            "geoNetwork_city", "geoNetwork_continent", 
            "geoNetwork_country", "geoNetwork_metro",
            "geoNetwork_networkDomain", "geoNetwork_region", 
            "geoNetwork_subContinent", "customDimensions_value"]
# str_cols = ["channelGrouping", 
#             "geoNetwork_city", "geoNetwork_continent", 
#             "geoNetwork_country", "geoNetwork_metro",
#             "geoNetwork_networkDomain", "geoNetwork_region", 
#             "geoNetwork_subContinent", "customDimensions_value"]
data[str_cols] = str(data[str_cols])
num_cols = ["year", "month","weekday","hour","minute","second","totals_hits", "totals_pageviews", "visitNumber", 'totals_bounces',  'totals_newVisits',
            'totals_transactions'] 
# num_cols = ["totals_hits", "totals_pageviews", "visitNumber", 'totals_bounces',  'totals_newVisits'] 


import torch.nn as nn
import torch.nn.functional
import torch.optim
import numpy.random
import math
import pandas
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

    
label_encoders = {}
for str_col in str_cols:
    label_encoders[str_col] = LabelEncoder()
    data[str_col] = label_encoders[str_col].fit_transform(data[str_col].astype(str))


testing_data = data
testing_id = testing_data['fullVisitorId']


net.eval()
label = testing_data['totals_transactionRevenue']
label = torch.cuda.FloatTensor(label.values)
testing = testing_data[str_cols + num_cols]
testing = torch.cuda.FloatTensor(testing.values)


testing = testing.unsqueeze(0)
testing = testing.permute(1,0,2)
label = label.unsqueeze(0)
label = label.permute(1,0)
output = net(testing)
criterion = torch.nn.MSELoss()
loss = torch.sqrt(criterion(output, label))
loss

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,31,40,43,44,45,48,49,61,68,69,70,75,77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


tensor(0.2992, device='cuda:0', grad_fn=<SqrtBackward>)

In [0]:
torch.save(net.state_dict(), './s1')

In [0]:
!ls

__MACOSX  new_data.csv	new_data.csv.zip  s1  sample_data


In [0]:
output

tensor([[ 0.0617],
        [-0.0368],
        [ 0.1882],
        ...,
        [-0.1321],
        [ 0.1062],
        [ 0.1176]], device='cuda:0', grad_fn=<ThAddmmBackward>)

In [0]:
from google.colab import files
files.download('./net')

In [0]:
net = torch.load('./net')

In [0]:
print(net)

CNN_net(
  (fc1): Sequential(
    (0): Conv1d(1, 16, kernel_size=(3,), stride=(1,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
  )
  (fc2): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (fc4): Linear(in_features=288, out_features=1, bias=True)
)
